In [68]:
#todo:implement fashion-mnist:

In [1]:
from zipfile import ZipFile
import os
import sys
import urllib
from model.model import Model
import urllib.request
import numpy as np
import cv2
from layers.layer_dense import Layer_Dense
from activation_func.relu import Activation_ReLU
from activation_func.softmax import Activation_Softmax
from loss.categorical_cross_entropy import Loss_CategoricalCrossentropy
from optimizers.adam import Optimizer_Adam
from accuracy.accuracy_categorical import Accuracy_Categorical


In [2]:
# #data preparation 
# URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
# FILE = 'fashion_mnist_images.zip'
# FOLDER = 'fashion_mnist_images'
# if not os.path.isfile(FILE):
#     print(f'Downloading {URL} and saving as {FILE}...')
#     urllib.request.urlretrieve(URL, FILE)

# print('Unzipping images...')
# with ZipFile(FILE) as zip_images:
#     zip_images.extractall(FOLDER) 

# print('Done!')

In [2]:
#load mnist dataset
def load_mnist_dataset(dataset,path):
    #list all directories and set them as label [0 to 9]
    labels = os.listdir(os.path.join(path, dataset))
    X = []
    y  = []
    
    for label in labels:
        for file in os.listdir(os.path.join(path,dataset,label)):
            image = cv2.imread(os.path.join(path, dataset, label, file),
                cv2.IMREAD_UNCHANGED)
                # And append it and a label to the lists
            X.append(image)
            y.append(label)

    return np.array(X),np.array(y).astype('uint8')

In [3]:
#create mnist dataset 
def create_data_mnist(path):
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

In [5]:
#load dataset
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')

FileNotFoundError: [Errno 2] No such file or directory: 'NNFS_BOOK/fashion_mnist_images/train'

In [6]:
# Shuffle the training dataset
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

X.shape,y.shape

((60000, 28, 28), (60000,))

In [7]:
# Scale and reshape samples
X = (X.reshape(X.shape[0], -1).astype(np.float32) - 127.5) / 127.5
X_test = (X_test.reshape(X_test.shape[0], -1).astype(np.float32) -
127.5) / 127.5

In [9]:
model_train = Model()
X.shape,X_test.shape

test


((60000, 784), (10000, 784))

In [10]:
#add layers to the list 
model_train.add(Layer_Dense(X.shape[1], 64))
model_train.add(Activation_ReLU())
model_train.add(Layer_Dense(64, 64))
model_train.add(Activation_ReLU())
model_train.add(Layer_Dense(64, 10))
model_train.add(Activation_Softmax())

In [11]:
#set loss,optimizer and accuracy  
model_train.set(
loss=Loss_CategoricalCrossentropy(),
optimizer=Optimizer_Adam(decay=5e-5),
accuracy=Accuracy_Categorical()
)

done


In [12]:
# Finalize the model
model_train.finalize()

In [13]:
model_train.train(X, y, validation_data=(X_test, y_test),
epochs=5, batch_size=128, print_every=100)

epoch1: 1
step: 0, acc: 0.078, loss: 2.303 (data_loss: 2.303, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.680, loss: 0.737 (data_loss: 0.737, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.727, loss: 0.740 (data_loss: 0.740, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.828, loss: 0.439 (data_loss: 0.439, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.859, loss: 0.395 (data_loss: 0.395, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.760, loss: 0.572 (data_loss: 0.572, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.718, loss: 0.753 (data_loss: 0.753, reg_loss: 0.000), lr: 0.0009771350400625367
validation, acc: 0.802, loss: 0.541
epoch1: 2
step: 0, acc: 0.828, loss: 0.477 (data_loss: 0.477, reg_loss: 0.000), lr: 0.0009770873027505008
step: 100, acc: 0.844, loss: 0.465 (data_loss: 0.465, reg_loss: 0.000), lr: 0.000972337012008362
step: 200, acc: 0.812, loss: 0.578 (data_loss: 0.578, reg_loss: 0.000), lr: 0.0009676